#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            print('BEAUTIFUL SOUP')
            if 'non trovato' in response2.text:
                #print('riprova')
                return 'non trovato'
            else :
                print('fare il codice per lo scraper dei risultati')       ######################################
        else :
            print('Qualcosa è andato storto')    

    else:
        print('ufficio non trovato')    

In [27]:
scraper(ufficio='TQD',ente='002',anno='2017',numero='3',natura='DI')

BEAUTIFUL SOUP
fare il codice per lo scraper dei risultati


#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [5]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

tipo = 'monitoria'

results = []
# mettere check attore in scraper (NO) oppure togliere if break e filtrare dopo i risultati (YEP) ?
# facciamo lo scraper di tutte le possibili combinazioni, alla fine filtreremo i risultati che ci interessano 
def tipo_scraper(tipo,ufficio,lista_enti,anno,numero):
    if tipo=='monitoria':
        for natura in list_natura_monitoria:
            for ente in lista_enti:
                result = scraper(ufficio,ente,anno,numero,natura)
                results.append(result)
    elif tipo == 'ppt':
        for natura in list_natura_ppt:
            for ente in lista_enti:
                result = scraper(ufficio,ente,anno,numero,natura)
                results.append(result)
    return results            


tipo_scraper('monitoria',ufficio='TQD',lista_enti=['002'],anno='2017',numero='3')

BEAUTIFUL SOUP
fare il codice per lo scraper dei risultati
BEAUTIFUL SOUP


#### sistemiamo l'input prima di mandare lo scaper 

In [120]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json

In [121]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [122]:
input = pd.read_excel('Input/Raw/Ricerca F24.xlsx')
input = clean(input)
input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
input = input.drop(columns=['RICERCA IQERA'])

mapping = {'albenga':'savona',
 'chiavenna':'sondrio',
 'cossato':'biella',
 'macomer':'nuoro',
 'isili':'nuoro',
 'chiaromonte': 'lagonegro',
 'orzinuovi' : 'brescia',
 'ovada': 'acquiterme',
 'pavullo nel frignano':'modena',
 'ponte san pietro':'bergamo',
 'romano di lombardia':'bergamo',
 'san benedetto del tronto':'ascoli piceno',
 'teano':'caserta',
 'venezia 2 - mestre' : 'venezia',
 'venosa':'potenza',
 'vimercate':'monza',
 'acerra' : 'casoria'
 }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input


In [123]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [124]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

In [125]:
input[input['code_office'].isna()]

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,nr.rg,year.rg,nr.decreto,year.decreto,nr.repertorio,year.repertorio,nr.crono,year.crono,nr.rgeppt,year.rgeppt,nr.repppt,year.repppt,tipo_ricerca,code,office


#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [126]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [127]:
enti_decoded['siracusa  ut dpsr']['tribunale']

['001', '007', '009', '012']

In [129]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)

In [130]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,nr.rg,year.rg,nr.decreto,year.decreto,nr.repertorio,year.repertorio,nr.crono,year.crono,nr.rgeppt,year.rgeppt,nr.repppt,year.repppt,tipo_ricerca,code,office,code_ente
0,AKCO300582766.001,CHIAVETTA CONO,tribunale,siracusa,AK NORDIC,7062,2017,588,2018,2263,2019,8245,2019,NaN,NaN,NaN,NaN,monitoria,TYN,siracusa ut dpsr,"[001, 007, 009, 012]"
1,AKCO300611623.001,ARIGO' PANCRAZIO,tribunale,siracusa,AK NORDIC,5067,2017,1931,2017,1981,2018,7606,2018,NaN,NaN,NaN,NaN,monitoria,TYN,siracusa ut dpsr,"[001, 007, 009, 012]"
2,CRS18966047,SCANZIANI MARIA JOSELLA,tribunale,monza,MBCREDIT,6996,2019,4293,2019,853,2021,1245,2021,NaN,NaN,NaN,NaN,monitoria,TNT,monza ut dpmb,[001]
3,CRS2294452,NUNNERI FRANCESCO,tribunale,napoli,MBCREDIT,14685,2020,7521,2020,3900,2021,4421,2021,NaN,NaN,NaN,NaN,monitoria,TER,napoli 1 ut dpna1,"[001, 003, 004, 018, 019, 020, 021, 022, 023, ..."
4,CRS2294452,NUNNERI FRANCESCO,tribunale,napoli,MBCREDIT,14685,2020,7521,2020,3900,2021,4421,2021,NaN,NaN,NaN,NaN,monitoria,TES,napoli 2 ut dpna1,[001]
5,CRS2294452,NUNNERI FRANCESCO,tribunale,napoli,MBCREDIT,14685,2020,7521,2020,3900,2021,4421,2021,NaN,NaN,NaN,NaN,monitoria,TET,napoli 3 ut dpna2,"[001, 002, 003, 004, 005, 006, 007, 016, 017, ..."
6,AKCO890006504896,GERARDO MASSA,tribunale,salerno,AK NORDIC,276,2017,343,2017,5966,2022,2844,2022,NaN,NaN,NaN,NaN,monitoria,TE3,salerno ut dpsa,"[001, 005, 020, 021, 022, 023, 024, 025, 026, ..."
7,CRS5606632,CIOTOLA PASQUALE,tribunale,napoli,MBCREDIT,10557,2021,4139,2021,10451,2021,11639,2021,NaN,NaN,NaN,NaN,monitoria,TER,napoli 1 ut dpna1,"[001, 003, 004, 018, 019, 020, 021, 022, 023, ..."
8,CRS5606632,CIOTOLA PASQUALE,tribunale,napoli,MBCREDIT,10557,2021,4139,2021,10451,2021,11639,2021,NaN,NaN,NaN,NaN,monitoria,TES,napoli 2 ut dpna1,[001]
9,CRS5606632,CIOTOLA PASQUALE,tribunale,napoli,MBCREDIT,10557,2021,4139,2021,10451,2021,11639,2021,NaN,NaN,NaN,NaN,monitoria,TET,napoli 3 ut dpna2,"[001, 002, 003, 004, 005, 006, 007, 016, 017, ..."


In [ ]:
# fare il mapping dell'ente 
# trovare il modo di fare i 4 / 2 numeri per tipo 
# tipo_scraper(tipo,ufficio,ente,anno,numero)
results =( input
    .apply(lambda row: tipo_scraper(tipo= row['tipo_ricerca'],ufficio= row['code'], ente= 'oo2', anno=row['year.rg'],numero= row['nr.rg']), axis=1)
)

# modificare come prendere ente     e numeri /anno 
